# Tic-Tac-Toe (Events + Observers)

This notebook uses event-style inputs with a `events` namespace.
Human plays X, AI plays O. The widget emits outputs, and Python responds by issuing events.

In [ ]:
import time
import vibe_widget as vw
import math
import random


# Create move prediction

In [5]:
def check_winner(board):
    # Winning combinations indices
    wins = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8), # Rows
        (0, 3, 6), (1, 4, 7), (2, 5, 8), # Cols
        (0, 4, 8), (2, 4, 6)             # Diagonals
    ]
    for a, b, c in wins:
        if board[a] == board[b] == board[c] and board[a] != 'b':
            return board[a]
    if 'b' not in board:
        return 'tie'
    return None

def minimax(board, depth, is_maximizing):
    result = check_winner(board)
    if result == 'o': return 10 - depth
    if result == 'x': return -10 + depth
    if result == 'tie': return 0

    if is_maximizing:
        best_score = -math.inf
        for i in range(9):
            if board[i] == 'b':
                board[i] = 'o'
                score = minimax(board, depth + 1, False)
                board[i] = 'b' # Backtrack
                best_score = max(score, best_score)
        return best_score
    else:
        best_score = math.inf
        for i in range(9):
            if board[i] == 'b':
                board[i] = 'x'
                score = minimax(board, depth + 1, True)
                board[i] = 'b' # Backtrack
                best_score = min(score, best_score)
        return best_score

# fn we'll trigger with an observer 
def pick_best_move(board_list):
    """
    Uses Minimax to find the optimal move. 
    Returns the index of the best move.
    """
    best_score = -math.inf
    best_moves = []
    
    # Check for immediate empty spots first to save time on empty board
    empty_spots = [i for i, x in enumerate(board_list) if x == 'b']
    
    # Optimization: If board is empty, pick center or corner to save recursion depth
    if len(empty_spots) == 9:
        return 4 # Center is usually best opener
    if len(empty_spots) == 8 and board_list[4] == 'b':
        return 4 # Take center if opponent didn't

    for i in empty_spots:
        # Try the move
        board_list[i] = 'o'
        score = minimax(board_list, 0, False)
        board_list[i] = 'b' # Undo the move

        if score > best_score:
            best_score = score
            best_moves = [i]
        elif score == best_score:
            best_moves.append(i)
            
    if best_moves:
        return random.choice(best_moves) # Randomize if multiple moves are equally perfect
    return None

## Observe outputs and issue events

In [6]:
game_board = vw.create(
    """Interactive Tic-Tac-Toe game board
    - Human plays X, AI plays O
    - Click cells to make moves
    - Outputs board_state, current_turn, game_over
    - Listen for custom events: ai_move(index: 0-8, row-major)
    - Handle events via model.on('msg:custom', ...) and apply ai_move
    - make sure to use a functional state update or a ref so the AI move applies on the latest board.
    """,
    outputs=vw.outputs(
        board_state="9-element array of 'x', 'o', or 'b'",
        game_over="boolean",
        current_turn="'x' or 'o'"
    ),
    actions=vw.actions(
        ai_move=vw.action(
            "index 0-8 (row-major) for the AI's move",
            params={"index": "0-8 row-major"}
        )
    ),
    cache=False
)

game_board


def on_turn_change(event):
    if event.new != "o":
        return

    # Allow the frontend to finish updating its state.
    time.sleep(0.1)

    board_state = game_board.outputs.board_state.value
    if not board_state or game_board.outputs.game_over.value:
        return

    if isinstance(board_state, str):
        import ast
        board_state = ast.literal_eval(board_state)

    board_list = list(board_state)
    if len(board_list) != 9:
        return

    move_index = pick_best_move(board_list) # can be replaced with a neural net
    if move_index is None:
        return

    game_board.actions.ai_move(index=move_index)


game_board.outputs.current_turn.observe(on_turn_change)


DynamicVibeWidget(board_state=<VibeExport Interactive Tic-Tac-Toe game board
    - Human plays X, AI plays O
 ...